In [10]:
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [11]:
from glob import  glob
from n2p_datamodule import Nuc2ProtDM
from n2p_dev import Nuc2Prot
import numpy as np

In [5]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose

# Setup

In [12]:
with initialize(version_base=None, config_path="config/"):
    cfg = compose(config_name="config")
    print(cfg)

MissingConfigException: Cannot find primary config 'config'. Check that it's in your config search path.

Config search path:
	provider=hydra, path=pkg://hydra.conf
	provider=main, path=file:///Users/amulyagarimella/Desktop/Buenrostro_Lab/intranuclear-structures/nucleus_to_protein_model/config
	provider=schema, path=structured://

In [ ]:
pdm = Nuc2ProtDM(
    images_path=config.data.images_path,
    labels_path=config.data.labels_path,
    sequences_path=config.data.sequences_path,
    batch_size=config.trainer.batch_size // 2,
    num_workers=config.trainer.num_workers,
    trim=config.data.trim,
    sequence_embedding=config.data.sequence_embedding,
    splits=config.splits,
)
pdm.setup()

In [ ]:
"""plm = Nuc2Prot
plm.eval()
plm.cuda()"""

# TODO

In [ ]:
dl = pdm.custom_dataloader(split_images='val', split_protein='train', shuffle=42)
batch = next(iter(dl))
true_images = batch['image'].detach().cpu().numpy()
batch['image'] = batch['image'].cuda()
predicted_images = []
predicted_latents = []
nis = 40
gs_range = np.linspace(1, 4, 11)

for gs in gs_range:
    predicted_latent = plm.sample(batch, guidance_scale=gs, cond_images=None, num_inference_steps=nis, seed=42)
    predicted_image = plm.autoencoder.decode(predicted_latent).sample
    predicted_images.append(predicted_image.detach().cpu().numpy())
    predicted_latents.append(predicted_latent.detach().cpu().numpy())

predicted_latents = np.stack(predicted_latents)
predicted_images = np.stack(predicted_images)

In [41]:
predicted_images = []
predicted_latents = []
# nis_range = np.linspace(20, 100, 7)
nis_range = np.linspace(50, 1000, 5)

gs = 3.0

for nis in nis_range:
    predicted_latent = plm.sample(batch, guidance_scale=gs, cond_images=None, num_inference_steps=int(nis), seed=42)
    predicted_image = plm.autoencoder.decode(predicted_latent).sample
    predicted_images.append(predicted_image.detach().cpu().numpy())
    predicted_latents.append(predicted_latent.detach().cpu().numpy())

predicted_latents = np.stack(predicted_latents)
predicted_images = np.stack(predicted_images)

NameError: name 'plm' is not defined